# Функциональный API

Мы хотим строить сложные разветвлённые модели, а не какие-то там жалкие линейные. Ещё хотим трюки разные использовать. Например, делать ResNet слои. В таких ситуациях нам на помощь приходит функциональный API от keras. Он позволяет придумывать и реализовывать сетки как угодно.

## 1. Введение в функциональный API

In [1]:
import tensorflow.keras as  keras
import tensorflow as tf
from tensorflow.keras import Input, layers 

input_tensor = Input(shape=(32,))
dense = layers.Dense(32, activation='relu')
output_tensor = dense(input_tensor)

Начнем с маленького примера, который описывает как обычную модель можно представить с помощью функционального API. На самом деле мы с вами уже видели его раньше. Зададим небольшую сетку с помощью класса `Sequential`. Она линейная. Каждый выход это вход для следующего. 

In [2]:
from tensorflow.keras.models import Sequential, Model 

seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,))) 
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

А теперь возьмём её и перепишем в функциональном стиле. Каждый слой это функция, которую мы применяем к какому-то входу. 

In [3]:
input_tensor = Input(shape=(64,))

x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [4]:
from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes = True)

# можно нарисовать картинку 
# SVG(model_to_dot(model,show_shapes=True).create(prog='dot', format='svg'))

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


Единственная часть, которая может показаться немного волшебной - создание экземпляра объекта `Model` с использованием только входного и выходного тензоров. За кулисами Keras извлекает все слои, участвующие в переходе от `input_tensor` к `output_tensor`, объединяя их в граф вычислений. Тут важно заметить, что `output_tensor` и `input_tensor` связаны межу собой вычислениями. Если попытаться построить модель из входных и выходных данных, которые не были связаны выскочит ошибка. 

In [5]:
unrelated_input = Input(shape=(32,))
bad_model = Model(unrelated_input, output_tensor)

Ошибка говорит нам о том, что керас не смог достичь `output`, преобразовывая `input`. Всё остальное работает точно также, как и раньше.

In [6]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10)) 

model.fit(x_train, y_train, epochs=10, batch_size=128)

score = model.evaluate(x_train, y_train, verbose=0)
score

Train on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 1s 617us/sample - loss: 12.4903
Epoch 2/10
1000/1000 [==============================] - 0s 31us/sample - loss: 14.9156
Epoch 3/10
1000/1000 [==============================] - 0s 27us/sample - loss: 18.3140
Epoch 4/10
1000/1000 [==============================] - 0s 54us/sample - loss: 21.7487
Epoch 5/10
1000/1000 [==============================] - 0s 26us/sample - loss: 25.5756
Epoch 6/10
1000/1000 [==============================] - 0s 14us/sample - loss: 29.6827
Epoch 7/10
1000/1000 [==============================] - 0s 36us/sample - loss: 33.7241
Epoch 8/10
1000/1000 [==============================] - 0s 12us/sample - loss: 38.3102
Epoch 9/10
1000/1000 [==============================] - 0s 44us/sample - loss: 41.8775
Epoch 10/10
1000/1000 [==============================] - 0s 36us/sample - loss: 46.1290


48.51844696044922


## 2. Модели с несколькими входами

Функциональный API может использоваться для построения моделей с несколькими входами. Как правило, такие модели в какой-то момент объединяют свои ветви, используя слой, который может объединять несколько тензоров. (`keras.layers.add`, `keras.layers.concatenate` и тп)

### Диалоговая система

Представим себе, что мы пытаемся построить сетку, которая умеет отвечать на вопросы. Чтобы выйчить её, надо показать ей кучу примеров вопросов и ответов на них. У сетки будет два входа. На выходе мы будем получать вероятность того, что ответ релевантен вопросу. 

In [7]:
from tensorflow.keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

# Ввод текста представляет собой последовательность целых чисел переменной длины.
# Обратите внимание, что вы можете по желанию назвать входы
text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(text_vocabulary_size,64)(text_input)

# С помощью LSTM переводим текст в последовательность
encoded_text = layers.LSTM(32)(embedded_text)

# Тот же процесс (с разными экземплярами слоя) для вопроса
question_input = Input(shape=(None,),dtype='int32',name='question')
embedded_question = layers.Embedding(question_vocabulary_size,32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question) 

# Соеденяем вопрос и текст
concatenated = layers.concatenate([encoded_text, encoded_question],
                                  axis=-1)
# Добавляем softmax итоговый
answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

# Модельс 2-мя входами и 1 выходом, поэтому задаем таким образом
model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['acc'])

In [8]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
question (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 32)     320000      question[0][0]                   
____________________________________________________________________________________________

In [33]:
plot_model(model, show_shapes = True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


### Обучение модели с несколькими входами

In [10]:
import numpy as np
num_samples = 1000 
max_length = 100

# модельные данные
text = np.random.randint(1, text_vocabulary_size,size=(num_samples, max_length))
text

array([[8054,  360, 4332, ..., 6235, 5528, 5878],
       [6102, 8217, 5513, ..., 6377, 6579, 6850],
       [3864, 6967, 4492, ..., 8542, 2155,  145],
       ...,
       [7360, 5886, 4331, ..., 8644, 1445, 6148],
       [6349, 7374, 6710, ..., 6095, 5026,  527],
       [9805, 8936, 8299, ..., 8923, 7435, 8762]])

In [11]:
question = np.random.randint(1, question_vocabulary_size,size=(num_samples, max_length))
question

array([[ 275, 3200, 9866, ..., 2845, 9866, 3427],
       [7107, 8775, 5426, ..., 1259,  635, 9621],
       [6509, 4273, 4708, ..., 8719, 4789, 4020],
       ...,
       [8821, 8929, 1657, ..., 2399, 8735, 4123],
       [7580, 5237, 7327, ..., 2264, 4803, 1518],
       [6172, 5188, 8463, ..., 7259,  649, 7707]])

In [12]:
# выход из сетки это релевантность ответа вопросу
answers = np.random.randint(answer_vocabulary_size, size=(num_samples))
answers = keras.utils.to_categorical(answers, answer_vocabulary_size)
answers

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

Подать данные на вход можно двумя способами. Либо как лист:

In [13]:
model.fit([text, question], answers, epochs=10, batch_size=128)

Train on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 4s 4ms/sample - loss: 6.2148 - acc: 0.0020
Epoch 2/10
1000/1000 [==============================] - 1s 609us/sample - loss: 6.1977 - acc: 0.0390
Epoch 3/10
1000/1000 [==============================] - 1s 724us/sample - loss: 6.1456 - acc: 0.0070
Epoch 4/10
1000/1000 [==============================] - 1s 754us/sample - loss: 6.0684 - acc: 0.0090
Epoch 5/10
1000/1000 [==============================] - 1s 698us/sample - loss: 5.9958 - acc: 0.0080
Epoch 6/10
1000/1000 [==============================] - 1s 706us/sample - loss: 5.8934 - acc: 0.0140
Epoch 7/10
1000/1000 [==============================] - 1s 844us/sample - loss: 5.7897 - acc: 0.0110
Epoch 8/10
1000/1000 [==============================] - 1s 890us/sample - loss: 5.7059 - acc: 0.0120
Epoch 9/10
1000/1000 [==============================] - 1s 762us/sample - loss: 5.6233 - acc: 0.0150
Epoch 10/10
1000/1000 [==============================] - 1s 865us/sampl

Либо как словарь:

In [14]:
model.fit({'text': text, 'question': question}, answers,epochs=10, batch_size=128)

Train on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 1s 926us/sample - loss: 5.4933 - acc: 0.0190
Epoch 2/10
1000/1000 [==============================] - 1s 890us/sample - loss: 5.4302 - acc: 0.0210
Epoch 3/10
1000/1000 [==============================] - 1s 758us/sample - loss: 5.3679 - acc: 0.0270
Epoch 4/10
1000/1000 [==============================] - 1s 665us/sample - loss: 5.3056 - acc: 0.0320
Epoch 5/10
1000/1000 [==============================] - 1s 650us/sample - loss: 5.2587 - acc: 0.0310
Epoch 6/10
1000/1000 [==============================] - 1s 649us/sample - loss: 5.2099 - acc: 0.0420
Epoch 7/10
1000/1000 [==============================] - 1s 638us/sample - loss: 5.1416 - acc: 0.0460
Epoch 8/10
1000/1000 [==============================] - 1s 665us/sample - loss: 5.1055 - acc: 0.0450
Epoch 9/10
1000/1000 [==============================] - 1s 656us/sample - loss: 5.0547 - acc: 0.0410
Epoch 10/10
1000/1000 [==============================] - 1s 661us/sam

## 3. Модели с несколькими выходами

Можно собирать модели с несколькими выходами! 

### Пример - прогноз возраста, пола и дохода от постов в социальных сетях

Сеть пытается предсказать по сообщению человека в социальных сетках его возраст, пол и доход. 

In [15]:
vocabulary_size = 50000 
num_income_groups = 10 

In [16]:
import numpy as np 

num_samples = 1000 
max_length = 100

posts = np.random.randint(1, vocabulary_size, size=(num_samples, max_length))
posts

array([[ 1455,  7155, 48861, ..., 44334, 16038, 12194],
       [ 2261,  4525, 45128, ...,  9752, 33206,  1588],
       [22284, 35175, 20730, ..., 17430, 11918, 24401],
       ...,
       [42345, 24480,  7447, ..., 33867, 31447, 39097],
       [27833, 44998, 29995, ..., 40870, 20602, 34144],
       [37888, 43211,   424, ...,   537, 24239, 30451]])

In [17]:
age_targets = np.random.randint(0, 100, size=(num_samples,1))
age_targets[:10]

array([[39],
       [58],
       [23],
       [81],
       [70],
       [14],
       [ 9],
       [66],
       [36],
       [60]])

In [18]:
income_targets = np.random.randint(1, num_income_groups, size=(num_samples,1))
income_targets = keras.utils.to_categorical(income_targets,num_income_groups)
income_targets[:10]

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]], dtype=float32)

In [19]:
gender_targets = np.random.randint(0, 2, size=(num_samples,1))
gender_targets[:10]

array([[1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0]])

Собираем модель.

In [20]:
posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocabulary_size,256)(posts_input)

x = layers.Conv1D(128, 5, activation='relu', padding='same')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu', padding='same')(x)
x = layers.Conv1D(256, 5, activation='relu', padding='same')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu', padding='same')(x)
x = layers.Conv1D(256, 5, activation='relu', padding='same')(x) 
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x) 

# Заметим, что выходным лаерам лучше дать имя.
age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups,
                                 activation='softmax',name='income')(x)

gender_prediction = layers.Dense(1, activation='sigmoid',
                                 name='gender')(x)

model = Model(posts_input,[age_prediction, income_prediction, 
                           gender_prediction])

Лоя разных выходов надо задавать разные функции потерь. Например, прогнозирование возраста регрессия, пола классификация. Все эти потери надо будет объединить в один функционал. Каждой придать свой вес. Сложно, короче говоря. 

In [21]:
# снова либо списком либо словарем, словарь работает если мы дали тензорам имена
model.compile(optimizer='rmsprop', loss=['mse', 
                                         'categorical_crossentropy',
                                         'binary_crossentropy'])

model.compile(optimizer='rmsprop',loss={'age': 'mse',
                                        'income': 'categorical_crossentropy',
                                        'gender': 'binary_crossentropy'})

Посмотрим как можно на разные функции потерь накидывать веса. Из-за того, что они могут быть несбалансированны, один выход может задоминировать другой. 

In [22]:
model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
              loss_weights=[0.25, 1., 10.]) 

model.compile(optimizer='rmsprop',
              
              loss={'age': 'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'},
              
              loss_weights={'age': 0.25,
                            'income': 1.,
                            'gender': 10.})

In [23]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 256)    12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, None, 128)    163968      embedding_2[0][0]                
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, None, 128)    0           conv1d[0][0]                     
____________________________________________________________________________________________

In [24]:
plot_model(model, show_shapes = True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


Подавать данные на вход снова можно двумя способами.

In [25]:
model.fit(posts, [age_targets, income_targets, gender_targets], epochs=10, batch_size=64)

Train on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 3s 3ms/sample - loss: 660.0024 - age_loss: 2471.4485 - income_loss: 4.6153 - gender_loss: 2.6309
Epoch 2/10
1000/1000 [==============================] - 2s 2ms/sample - loss: 155.6189 - age_loss: 572.9353 - income_loss: 2.8619 - gender_loss: 0.9811
Epoch 3/10
1000/1000 [==============================] - 2s 2ms/sample - loss: 83.9520 - age_loss: 276.7404 - income_loss: 2.3599 - gender_loss: 1.1944
Epoch 4/10
1000/1000 [==============================] - 2s 2ms/sample - loss: 114.6498 - age_loss: 398.0174 - income_loss: 2.3626 - gender_loss: 1.1643
Epoch 5/10
1000/1000 [==============================] - 2s 2ms/sample - loss: 98.4076 - age_loss: 332.9329 - income_loss: 2.3560 - gender_loss: 1.1483
Epoch 6/10
1000/1000 [==============================] - 1s 1ms/sample - loss: 113.3189 - age_loss: 421.8441 - income_loss: 2.3374 - gender_loss: 0.9052
Epoch 7/10
1000/1000 [==============================] - 1s 1ms/samp

In [26]:
model.fit(posts, {'age': age_targets,
                  'income': income_targets,
                  'gender': gender_targets},
         epochs=10, 
         batch_size=64)

Train on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 2s 2ms/sample - loss: 57.2643 - age_loss: 181.5103 - income_loss: 2.3440 - gender_loss: 0.9099
Epoch 2/10
1000/1000 [==============================] - 1s 1ms/sample - loss: 78.5701 - age_loss: 269.6850 - income_loss: 2.3117 - gender_loss: 0.8405
Epoch 3/10
1000/1000 [==============================] - 2s 2ms/sample - loss: 80.1113 - age_loss: 261.4580 - income_loss: 2.3645 - gender_loss: 1.1071
Epoch 4/10
1000/1000 [==============================] - 2s 2ms/sample - loss: 62.6108 - age_loss: 194.2973 - income_loss: 2.3400 - gender_loss: 1.1605
Epoch 5/10
1000/1000 [==============================] - 2s 2ms/sample - loss: 66.7655 - age_loss: 218.3465 - income_loss: 2.3678 - gender_loss: 1.0616
Epoch 6/10
1000/1000 [==============================] - 2s 2ms/sample - loss: 77.5890 - age_loss: 269.5192 - income_loss: 2.3662 - gender_loss: 0.9126 1s - loss: 120.4092 - age_loss: 426.7166 - income_loss: 2.3031 - gend
Ep

## 4 Направленные ациклические графы

Можно собирать сетки со сложной внутренней топологией. Keras разрешает ориентровать слои как угодно. Главноe, чтобы не было циклов. 

### 4.1 Inception модули

Ну чтож ваша очередь объяснять, что тут в коде происходит :) не только же мне мучаться:)

In [27]:
# Пример реализован для MNIST 

x = Input(shape=(28, 28, 1), dtype='float32', name='images')

print("x.shape:",x.shape)


branch_a = layers.Conv2D(128, 1, padding='same',
                         activation='relu', strides=2)(x)


branch_b = layers.Conv2D(128, 1, padding='same', 
                         activation='relu')(x)
branch_b = layers.Conv2D(128, 3, padding='same',
                         activation='relu', strides=2)(branch_b)


branch_c = layers.AveragePooling2D(3,  padding='same', strides=2)(x)
branch_c = layers.Conv2D(128, 3, 
                         padding='same',
                         activation='relu')(branch_c)

branch_d = layers.Conv2D(128, 1, padding='same', activation='relu')(x) 
branch_d = layers.Conv2D(128, 3, padding='same', activation='relu')(branch_d)
branch_d = layers.Conv2D(128, 3, padding='same', activation='relu', strides=2)(branch_d)


output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], 
                            axis=-1)

output = layers.Flatten()(output)
output = layers.Dense(512, activation='relu')(output)
predictions = layers.Dense(10, activation='softmax')(output)

model_intersept = keras.models.Model(inputs=x, outputs=predictions)
model.compile(optimizer=keras.optimizers.RMSprop(lr=2e-3, decay=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

x.shape: (None, 28, 28, 1)


In [28]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 256)    12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, None, 128)    163968      embedding_2[0][0]                
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, None, 128)    0           conv1d[0][0]                     
____________________________________________________________________________________________

In [32]:
plot_model(model, show_shapes = True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


### 4.2  ResNET

Теперь очередь Resnet-модуля :) 

In [40]:
from tensorflow.keras import layers 
from tensorflow.keras.layers import Input


# Этот пример ожидаем 4д тензор
# Возвращаем тензор, который ожидаем для MNIST
x = Input(shape=(28, 28, 1), dtype='float32', name='images')
print("x.shape:",x.shape)

# Применяем преобразования
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)

# Добавляем х вконец
output = layers.add([y, x])

# Добавляем сверху классификатор
output = layers.Flatten()(output)
output = layers.Dense(512, activation='relu')(output)
predictions = layers.Dense(10, activation='softmax')(output)
model = keras.models.Model(inputs=x, outputs=predictions)

x.shape: (None, 28, 28, 1)


In [41]:
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
images (InputLayer)             [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 28, 28, 128)  1280        images[0][0]                     
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 28, 28, 128)  147584      conv2d_7[0][0]                   
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 28, 28, 128)  147584      conv2d_8[0][0]                   
____________________________________________________________________________________________

In [42]:
plot_model(model, show_shapes = True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


## 5. Модели как слои

В слои можно заворачивать целые модели.

    у = модель (х)

Если модель имеет несколько входных тензоров и несколько выходных тензоров, ее следует вызывать со списком тензоров:

    y1, y2 = модель ([x1, x2])

Когда вы вызываете экземпляр модели, вы повторно используете вес модели - точно так же, как и при вызове экземпляра слоя. Вызов экземпляра, будь то экземпляр уровня или экземпляра модели, всегда будет повторно использовать существующие изученные представления экземпляра, что интуитивно понятно.

In [43]:
from tensorflow.keras import applications 


nbr_classes = 10

# Возьмем готовую модель Xception
xception_base = applications.Xception(weights=None,include_top=False)

# входная картинка  250 × 250 RGB.
left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

left_features = xception_base(left_input)
right_features = xception_base(right_input)

merged_features = layers.concatenate([left_features, right_features], axis=-1)

predictions = layers.Dense(nbr_classes, activation='softmax')(merged_features)

#  Собираем нашу модельку
model = Model([left_input, right_input], predictions)

In [44]:
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 250, 250, 3) 0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 250, 250, 3) 0                                            
__________________________________________________________________________________________________
xception (Model)                multiple             20861480    input_4[0][0]                    
                                                                 input_5[0][0]                    
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 8, 8, 4096)   0           xception[1][0]             

In [45]:
plot_model(model, show_shapes = True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


Итого - давайте теперь соберем модель с двумя входами, где оба входа - обработчики картинок.
Выхода у нас тоже 2 - предположим мы решаем задачу следующего ввида - ищем как контент картинки, так и время года!

#### Ну и на последок 
 мы же все помним, мы можем писать кастомные callback. Теперь поэтапно смотрим на эту прелесть.
 
 Возьмем уже известную вам модельку с прошлой пары и будем модифицировать наши результаты.

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
keras, L = tf.keras, tf.keras.layers

import numpy as np
import random
from tqdm import tqdm

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline


fashion_mnist = tf.keras.datasets.fashion_mnist

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

### ВАШ код предобработки данных ###


class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

Теперь давайте соберём свёртоную сеть: 

* Свёртка с ядром $5 \times 5$, same padding и $32$ каналами
* ReLU
* Макспулинг размера $2 \times 2$
* Свёртка с ядром $5 \times 5$ и $16$ каналами  и same padding
* ReLU
* Макспулинг размера $2 \times 2$ с шагом (strides) $2$ по обеим осям 
* Дальше сделайте `Flatten` и сделайте два полносвязных слоя с ReLU и $120$ и $60$ нейронами

In [ ]:
model_1 = Sequential( )

### и тут

model_1.compile("adam", "categorical_crossentropy", metrics=["categorical_accuracy"])

hist = model_1.fit(X_train, y_train, validation_split= 0.2,
                        batch_size=500, epochs=3, verbose=1)

In [ ]:
# Наш класс должен быть отнаследован от керасовского класса

class MyCustomCallback(tf.keras.callbacks.Callback):
    pass

# Но мы можем переписать методы, которые мы отнаследовали
# но для начала посмотреть на них

In [ ]:
dir(tf.keras.callbacks.Callback)

In [ ]:
# Теперь нам надо переписать методы которые нам потребуются
# А задача в следующем - посмотреть какой класс проседает по точности на валидационных данных


In [ ]:
precision_my = MyCustomCallback........

In [ ]:
hist = model_1.fit(X_train, y_train, validation_split= 0.2,epochs=1,
                       callbacks=[precision_my])